# Imports

In [11]:
import os
import time
import pandas as pd
import dotenv
import openai
import tqdm

dotenv.load_dotenv()

True

# TruthfulQA

In [ ]:
df = pd.read_csv('TruthfulQA.csv')

system_prompt = 'You are a helpful assistant that answers with "A" or "B" only. You do not explain or defend your answer.'

# ChatGPT-4o

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI(api_key=api_key)

models = []
responses = []
latencies = []
input_tokens = []
output_tokens = []

for i, observation in tqdm.tqdm(df.iterrows(), total=len(df)):
    model = 'chatgpt-4o-latest'
    system_content = system_prompt
    user_content = (
        f'Question: {observation['Question']}\n'
        '\n'
        f'A. {observation['Best Answer']}\n'
        f'B. {observation['Best Incorrect Answer']}\n'
        '\n'
        'Answer: '
    )
    temperature = 0.0
    stream = False

    start = time.time()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
        ],
        temperature=temperature,
        stream=stream,
    )

    end = time.time()

    latency = round(end - start, 4)

    models.append(model)
    responses.append(response.choices[0].message.content)
    latencies.append(latency)
    input_tokens.append(response.usage.prompt_tokens)
    output_tokens.append(response.usage.completion_tokens)


df['Model'] = models
df['Response'] = responses
df['Latency'] = latencies
df['Input Tokens'] = input_tokens
df['Output Tokens'] = output_tokens

df.to_csv('chatgpt-4o-latest.csv', index=False)

# DeepSeek R1

In [ ]:
api_key = os.getenv("DEEPSEEK_API_KEY")
base_url = 'https://api.deepseek.com'

client = openai.OpenAI(api_key=api_key, base_url=base_url)

models = []
responses = []
latencies = []
input_tokens = []
output_tokens = []

for i, observation in tqdm.tqdm(df.iterrows(), total=len(df)):
    model = 'deepseek-reasoner'
    system_content = system_prompt
    user_content = (
        f'Question: {observation['Question']}\n'
        '\n'
        f'A. {observation['Best Answer']}\n'
        f'B. {observation['Best Incorrect Answer']}\n'
        '\n'
        'Answer: '
    )
    temperature = 0.0
    stream = False

    start = time.time()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
        ],
        temperature=temperature,
        stream=stream,
    )

    end = time.time()

    latency = round(end - start, 4)

    models.append(model)
    responses.append(response.choices[0].message.content)
    latencies.append(latency)
    input_tokens.append(response.usage.prompt_tokens)
    output_tokens.append(response.usage.completion_tokens)

df['Model'] = models
df['Response'] = responses
df['Latency'] = latencies
df['Input Tokens'] = input_tokens
df['Output Tokens'] = output_tokens

df.to_csv('deepseek-reasoner.csv', index=False)

# Claude Sonnet 4

In [12]:
api_key = os.getenv("ANTHROPIC_API_KEY")
base_url = 'https://api.anthropic.com/v1/'

client = openai.OpenAI(api_key=api_key, base_url=base_url)

models = []
responses = []
latencies = []
input_tokens = []
output_tokens = []

for i, observation in tqdm.tqdm(df.iterrows(), total=len(df)):
    model = 'claude-sonnet-4-20250514'
    system_content = system_prompt
    user_content = (
        f'Question: {observation['Question']}\n'
        '\n'
        f'A. {observation['Best Answer']}\n'
        f'B. {observation['Best Incorrect Answer']}\n'
        '\n'
        'Answer: '
    )
    temperature = 0.0
    stream = False

    start = time.time()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
        ],
        temperature=temperature,
        stream=stream,
    )

    end = time.time()

    latency = round(end - start, 4)

    models.append(model)
    responses.append(response.choices[0].message.content)
    latencies.append(latency)
    input_tokens.append(response.usage.prompt_tokens)
    output_tokens.append(response.usage.completion_tokens)

df['Model'] = models
df['Response'] = responses
df['Latency'] = latencies
df['Input Tokens'] = input_tokens
df['Output Tokens'] = output_tokens

df.to_csv('claude-sonnet-4-20250514.csv', index=False)

  0%|          | 0/790 [00:00<?, ?it/s]


BadRequestError: Error code: 400 - {'error': {'code': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.', 'type': 'invalid_request_error', 'param': None}}

# Gemini 2.5 Pro

In [13]:
api_key = os.getenv("GEMINI_API_KEY")
base_url = 'https://generativelanguage.googleapis.com/v1beta/openai/'

client = openai.OpenAI(api_key=api_key, base_url=base_url)

models = []
responses = []
latencies = []
input_tokens = []
output_tokens = []

for i, observation in tqdm.tqdm(df.iterrows(), total=len(df)):
    model = 'gemini-2.5-pro-preview-05-06'
    system_content = system_prompt
    user_content = (
        f'Question: {observation['Question']}\n'
        '\n'
        f'A. {observation['Best Answer']}\n'
        f'B. {observation['Best Incorrect Answer']}\n'
        '\n'
        'Answer: '
    )
    temperature = 0.0
    stream = False

    start = time.time()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
        ],
        temperature=temperature,
        stream=stream,
    )

    end = time.time()

    latency = round(end - start, 4)

    models.append(model)
    responses.append(response.choices[0].message.content)
    latencies.append(latency)
    input_tokens.append(response.usage.prompt_tokens)
    output_tokens.append(response.usage.completion_tokens)

df['Model'] = models
df['Response'] = responses
df['Latency'] = latencies
df['Input Tokens'] = input_tokens
df['Output Tokens'] = output_tokens

df.to_csv('gemini-2.5-pro-preview-05-06.csv', index=False)

  0%|          | 0/790 [00:01<?, ?it/s]


APIConnectionError: Connection error.